## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Nov. 10th Nov. 17th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,November-15-2024,Condo/Co-op,480 High Point Dr Unit B,Delray Beach,FL,33445.0,160000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/480-Hig...,Beaches MLS,RX-11016450,N,Y,26.463064,-80.100238


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,2024-11-15,Condo/Co-op,480 High Point Dr Unit B,Delray Beach,FL,33445.0,160000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/480-Hig...,Beaches MLS,RX-11016450,N,Y,26.463064,-80.100238
13,PAST SALE,2024-11-15,Condo/Co-op,14442 Via Royale #1,Delray Beach,FL,33446.0,335000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/14442-V...,Beaches MLS,RX-11018971,N,Y,26.464236,-80.167569
2,PAST SALE,2024-11-15,Condo/Co-op,3050 Norwood Pl Unit N109,Boca Raton,FL,33431.0,305000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/3050-Norw...,Beaches MLS,RX-11027067,N,Y,26.379380,-80.087400
25,PAST SALE,2024-11-15,Condo/Co-op,1801 Belmont Pl,Boynton Beach,FL,33436.0,255000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/1801-B...,Beaches MLS,RX-11021517,N,Y,26.517359,-80.097997
33,PAST SALE,2024-11-15,Condo/Co-op,691 Burgundy O,Delray Beach,FL,33484.0,135000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/691-Bur...,Beaches MLS,RX-11005384,N,Y,26.445271,-80.131475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,PAST SALE,2024-11-12,Condo/Co-op,3630 Gardens Pkwy Unit 902c,Palm Beach Gardens,FL,33410.0,735000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/3...,Beaches MLS,RX-11029474,N,Y,26.849669,-80.090815
39,PAST SALE,2024-11-12,Condo/Co-op,18 Lexington Ln W Unit E,Palm Beach Gardens,FL,33418.0,405000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/1...,Beaches MLS,RX-10998285,N,Y,26.813946,-80.137755
41,PAST SALE,2024-11-12,Condo/Co-op,804 E Windward Way #601,Lantana,FL,33462.0,384000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lantana/804-E-Windwa...,Beaches MLS,RX-10985331,N,Y,26.591269,-80.050711
32,PAST SALE,2024-11-12,Condo/Co-op,95 Pelican Pointe Dr #102,Delray Beach,FL,33483.0,300000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/95-Peli...,Beaches MLS,F10455771,N,Y,26.423108,-80.072249


In [10]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [11]:
df_top_ten = df.head(10)

## Collect Agent Information

In [12]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [13]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [14]:
response_df = pd.DataFrame(response_list)

In [15]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [16]:
df_top_ten = merged_df

## Current Week's Values

In [17]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

62


In [18]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$809,035


In [19]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$397


In [20]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$50,160,200


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [21]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Palm Beach County
Input Previous Week Condo Sales Total (number units sold):
65
Input Previous Week Condo Average Sales Price:
427088
Input Previous Week Condo Average PSF:
297
Input Previous Week Condo Sales total (ex: 198_000_000)
27_800_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [22]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [23]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [24]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [25]:
pd.set_option('display.max_columns',None)

In [26]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-13,Condo/Co-op,2 N Breakers Row Unit N-42,Palm Beach,FL,33480.0,12500000.0,3.0,3.5,N/a,3129.0,NaN,1986.0,NaN,3995.0,15487.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Beaches MLS,RX-10939522,N,Y,26.71714,-80.034673,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Kevin Condon,"Sotheby's Intl. Realty, Inc.",None,None,Suzanne Trapani Frisbie,The Corcoran Group,None,None,1,orange


In [27]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [28]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-13,Condo/Co-op,2 N Breakers Row Unit N-42,Palm Beach,FL,33480.0,12500000.0,3.0,3.5,N/a,3129.0,NaN,1986.0,NaN,3995.0,15487.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Beaches MLS,RX-10939522,N,Y,26.717140,-80.034673,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Kevin Condon,"Sotheby's Intl. Realty, Inc.",None,None,Suzanne Trapani Frisbie,The Corcoran Group,None,None,1,orange
1,PAST SALE,2024-11-14,Condo/Co-op,1901 S Ocean Blvd #19,Delray Beach,FL,33483.0,10796000.0,4.0,5.5,Ocean Delray Condo,3841.0,180.0,2021.0,NaN,2811.0,8097.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1901-S-...,Beaches MLS,RX-11021711,N,Y,26.437931,-80.062238,https://www.redfin.com/FL/Delray-Beach/1901-S-...,Jennifer Kilpatrick,The Corcoran Group,Mickel Averick,The Corcoran Group,Todd David Lipsich,Luxury Partners Realty,None,None,2,blue
2,PAST SALE,2024-11-14,Condo/Co-op,2100 S Ocean Blvd Unit 407s,Palm Beach,FL,33480.0,4050000.0,2.0,2.5,Twenty One Hundred Condo,2385.0,NaN,1980.0,NaN,1698.0,3034.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2100-S-Oc...,Beaches MLS,RX-11017997,N,Y,26.642292,-80.038181,https://www.redfin.com/FL/Palm-Beach/2100-S-Oc...,Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,Adam Zimmerman,Douglas Elliman (Palm Beach),Jacqueline Zimmerman,Douglas Elliman (Palm Beach),3,blue
3,PAST SALE,2024-11-12,Condo/Co-op,700 Ocean Royale Way #1201,Juno Beach,FL,33408.0,2175000.0,3.0,3.5,Ocean Royale Condo,2936.0,NaN,2000.0,NaN,741.0,2900.0,Sold,NaN,NaN,https://www.redfin.com/FL/Juno-Beach/700-Ocean...,Beaches MLS,RX-11020844,N,Y,26.891690,-80.058145,https://www.redfin.com/FL/Juno-Beach/700-Ocean...,Holly Meyer Lucas,"Compass Florida, LLC (Jupiter)",Muriel Paton,"Compass Florida, LLC (Jupiter)",Holly Meyer Lucas,"Compass Florida, LLC (Jupiter)",Jenifer Greever,"Compass Florida, LLC (Jupiter)",4,blue
4,PAST SALE,2024-11-13,Condo/Co-op,1225 S Ocean Blvd #306,Delray Beach,FL,33483.0,1850000.0,2.0,2.5,Banyan House Condo,1911.0,NaN,1967.0,NaN,968.0,1783.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1225-S-...,Beaches MLS,RX-11026097,N,Y,26.443001,-80.061098,https://www.redfin.com/FL/Delray-Beach/1225-S-...,Candace Friis,The Corcoran Group,Phillip Friis,The Corcoran Group,Candace Friis,The Corcoran Group,None,None,5,blue
5,PAST SALE,2024-11-12,Condo/Co-op,1225 S Ocean Blvd #406,Delray Beach,FL,33483.0,1725000.0,2.0,2.0,Banyan House Condo,1911.0,NaN,1967.0,NaN,903.0,1783.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1225-S-...,Beaches MLS,RX-11021598,N,Y,26.443001,-80.061098,https://www.redfin.com/FL/Delray-Beach/1225-S-...,Michael O'Connor,Douglas Elliman,Nicholas Malinosky,Douglas Elliman,Candace Friis,The Corcoran Group,Phillip Friis,The Corcoran Group,6,blue
6,PAST SALE,2024-11-12,Condo/Co-op,801 S Olive Ave #1604,West Palm Beach,FL,33401.0,1500000.0,2.0,2.0,One City Plaza Condominium,1475.0,NaN,2006.0,NaN,1017.0,1570.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/801-...,Beaches MLS,RX-10980984,N,Y,26.705820,-80.052488,https://www.redfin.com/FL/West-Palm-Beach/801-...,Kevin Oh,KW Reserve Palm Beach,None,None,Ginger Gowing,Luxury Real Estate Group LLC,None,None,7,blue
7,PAST SALE,2024-11-14,Condo/Co-op,2000 S Ocean Blvd Unit 5-E,Boca Raton,FL,33432.0,1140000.0,2.0,2.0,Whitehall,1236.0,NaN,1971.0,NaN,922.0,1467.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2000-S-Oc...,Beaches MLS,RX-11022495,N,Y,26.328302,-80.074423,https://www.redfin.com/FL/Boca-Raton/2000-S-Oc...,Gregory Lynn,Lynn Realty Group,Charli Lynn,

In [29]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'Nov. 3rd — Nov. 10th')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [30]:
m.save('index.html')

## Data snagger

In [31]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [32]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-13,Condo/Co-op,2 N Breakers Row Unit N-42,Palm Beach,FL,33480.0,12500000.0,3.0,3.5,N/a,3129.0,NaN,1986.0,NaN,3995.0,15487.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Beaches MLS,RX-10939522,N,Y,26.71714,-80.034673,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Kevin Condon,"Sotheby's Intl. Realty, Inc.",None,None,Suzanne Trapani Frisbie,The Corcoran Group,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [33]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_11182024


# CREATE TEMPLATE 

In [34]:
muni_set = set(df_top_ten['CITY'])

In [35]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [36]:
df_top_ten.reset_index(inplace=True,drop=True)

In [37]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [38]:
top_sale

'N/a at 2 N Breakers Row Unit N-42 in Palm Beach'

In [39]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [40]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [41]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-13,Condo/Co-op,2 N Breakers Row Unit N-42,Palm Beach,FL,33480.0,12500000.0,3.0,3.5,N/a,3129.0,NaN,1986.0,NaN,3995.0,15487.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Beaches MLS,RX-10939522,N,Y,26.717140,-80.034673,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Kevin Condon,"Sotheby's Intl. Realty, Inc.",None,None,Suzanne Trapani Frisbie,The Corcoran Group,None,None,1,orange,N/a at 2 N Breakers Row Unit N-42 in Palm Beach
1,PAST SALE,2024-11-14,Condo/Co-op,1901 S Ocean Blvd #19,Delray Beach,FL,33483.0,10796000.0,4.0,5.5,Ocean Delray Condo,3841.0,180.0,2021.0,NaN,2811.0,8097.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1901-S-...,Beaches MLS,RX-11021711,N,Y,26.437931,-80.062238,https://www.redfin.com/FL/Delray-Beach/1901-S-...,Jennifer Kilpatrick,The Corcoran Group,Mickel Averick,The Corcoran Group,Todd David Lipsich,Luxury Partners Realty,None,None,2,blue,Ocean Delray Condo at 1901 S Ocean Blvd #19 in...
2,PAST SALE,2024-11-14,Condo/Co-op,2100 S Ocean Blvd Unit 407s,Palm Beach,FL,33480.0,4050000.0,2.0,2.5,Twenty One Hundred Condo,2385.0,NaN,1980.0,NaN,1698.0,3034.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2100-S-Oc...,Beaches MLS,RX-11017997,N,Y,26.642292,-80.038181,https://www.redfin.com/FL/Palm-Beach/2100-S-Oc...,Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,Adam Zimmerman,Douglas Elliman (Palm Beach),Jacqueline Zimmerman,Douglas Elliman (Palm Beach),3,blue,Twenty One Hundred Condo at 2100 S Ocean Blvd ...
3,PAST SALE,2024-11-12,Condo/Co-op,700 Ocean Royale Way #1201,Juno Beach,FL,33408.0,2175000.0,3.0,3.5,Ocean Royale Condo,2936.0,NaN,2000.0,NaN,741.0,2900.0,Sold,NaN,NaN,https://www.redfin.com/FL/Juno-Beach/700-Ocean...,Beaches MLS,RX-11020844,N,Y,26.891690,-80.058145,https://www.redfin.com/FL/Juno-Beach/700-Ocean...,Holly Meyer Lucas,"Compass Florida, LLC (Jupiter)",Muriel Paton,"Compass Florida, LLC (Jupiter)",Holly Meyer Lucas,"Compass Florida, LLC (Jupiter)",Jenifer Greever,"Compass Florida, LLC (Jupiter)",4,blue,Ocean Royale Condo at 700 Ocean Royale Way #12...
4,PAST SALE,2024-11-13,Condo/Co-op,1225 S Ocean Blvd #306,Delray Beach,FL,33483.0,1850000.0,2.0,2.5,Banyan House Condo,1911.0,NaN,1967.0,NaN,968.0,1783.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1225-S-...,Beaches MLS,RX-11026097,N,Y,26.443001,-80.061098,https://www.redfin.com/FL/Delray-Beach/1225-S-...,Candace Friis,The Corcoran Group,Phillip Friis,The Corcoran Group,Candace Friis,The Corcoran Group,None,None,5,blue,Banyan House Condo at 1225 S Ocean Blvd #306 i...
5,PAST SALE,2024-11-12,Condo/Co-op,1225 S Ocean Blvd #406,Delray Beach,FL,33483.0,1725000.0,2.0,2.0,Banyan House Condo,1911.0,NaN,1967.0,NaN,903.0,1783.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1225-S-...,Beaches MLS,RX-11021598,N,Y,26.443001,-80.061098,https://www.redfin.com/FL/Delray-Beach/1225-S-...,Michael O'Connor,Douglas Elliman,Nicholas Malinosky,Douglas Elliman,Candace Friis,The Corcoran Group,Phillip Friis,The Corcoran Group,6,blue,Banyan House Condo at 1225 S Ocean Blvd #406 i...
6,PAST SALE,2024-11-12,Condo/Co-op,801 S Olive Ave #1604,West Palm Beach,FL,33401.0,1500000.0,2.0,2.0,One City Plaza Condominium,1475.0,NaN,2006.0,NaN,1017.0,1570.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/801-...,Beaches MLS,RX-10980984,N,Y,26.705820,-80.052488,https://www.redfin.com/FL/West-Palm-Beach/801-...,Kevin Oh,KW Reserve Palm Beach,None,None,Ginger Gowing,Luxury Real Estate Group LLC,None,None,7,blue,One City Plaza Condominium at 801 S Olive Av

In [42]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: N/a closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $735,000M to $12,500,000M
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Palm Beach County condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, North Palm Beach, Palm Beach Gardens, Boca Raton, Palm Beach, West Palm Beach, Delray Beach, Juno Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 62 condo sales totaling $50,160,200 million from Nov. 10th to Nov. 17th. The previous week, brokers closed 65 condo sales totaling $27,800,000.

Last week’s units sold for an average of $809,035, higher tha

In [43]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [44]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [45]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [46]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-13,Condo/Co-op,2 N Breakers Row Unit N-42,Palm Beach,FL,33480.0,12500000.0,3.0,3.5,N/a,3129.0,NaN,1986.0,NaN,3995.0,15487.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Beaches MLS,RX-10939522,N,Y,26.717140,-80.034673,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Kevin Condon,"Sotheby's Intl. Realty, Inc.",None,None,Suzanne Trapani Frisbie,The Corcoran Group,None,None,1,orange,N/a at 2 N Breakers Row Unit N-42 in Palm Beach
1,PAST SALE,2024-11-14,Condo/Co-op,1901 S Ocean Blvd #19,Delray Beach,FL,33483.0,10796000.0,4.0,5.5,Ocean Delray Condo,3841.0,180.0,2021.0,NaN,2811.0,8097.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1901-S-...,Beaches MLS,RX-11021711,N,Y,26.437931,-80.062238,https://www.redfin.com/FL/Delray-Beach/1901-S-...,Jennifer Kilpatrick,The Corcoran Group,Mickel Averick,The Corcoran Group,Todd David Lipsich,Luxury Partners Realty,None,None,2,blue,Ocean Delray Condo at 1901 S Ocean Blvd #19 in...
2,PAST SALE,2024-11-14,Condo/Co-op,2100 S Ocean Blvd Unit 407s,Palm Beach,FL,33480.0,4050000.0,2.0,2.5,Twenty One Hundred Condo,2385.0,NaN,1980.0,NaN,1698.0,3034.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2100-S-Oc...,Beaches MLS,RX-11017997,N,Y,26.642292,-80.038181,https://www.redfin.com/FL/Palm-Beach/2100-S-Oc...,Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,Adam Zimmerman,Douglas Elliman (Palm Beach),Jacqueline Zimmerman,Douglas Elliman (Palm Beach),3,blue,Twenty One Hundred Condo at 2100 S Ocean Blvd ...
3,PAST SALE,2024-11-12,Condo/Co-op,700 Ocean Royale Way #1201,Juno Beach,FL,33408.0,2175000.0,3.0,3.5,Ocean Royale Condo,2936.0,NaN,2000.0,NaN,741.0,2900.0,Sold,NaN,NaN,https://www.redfin.com/FL/Juno-Beach/700-Ocean...,Beaches MLS,RX-11020844,N,Y,26.891690,-80.058145,https://www.redfin.com/FL/Juno-Beach/700-Ocean...,Holly Meyer Lucas,"Compass Florida, LLC (Jupiter)",Muriel Paton,"Compass Florida, LLC (Jupiter)",Holly Meyer Lucas,"Compass Florida, LLC (Jupiter)",Jenifer Greever,"Compass Florida, LLC (Jupiter)",4,blue,Ocean Royale Condo at 700 Ocean Royale Way #12...
4,PAST SALE,2024-11-13,Condo/Co-op,1225 S Ocean Blvd #306,Delray Beach,FL,33483.0,1850000.0,2.0,2.5,Banyan House Condo,1911.0,NaN,1967.0,NaN,968.0,1783.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1225-S-...,Beaches MLS,RX-11026097,N,Y,26.443001,-80.061098,https://www.redfin.com/FL/Delray-Beach/1225-S-...,Candace Friis,The Corcoran Group,Phillip Friis,The Corcoran Group,Candace Friis,The Corcoran Group,None,None,5,blue,Banyan House Condo at 1225 S Ocean Blvd #306 i...
5,PAST SALE,2024-11-12,Condo/Co-op,1225 S Ocean Blvd #406,Delray Beach,FL,33483.0,1725000.0,2.0,2.0,Banyan House Condo,1911.0,NaN,1967.0,NaN,903.0,1783.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1225-S-...,Beaches MLS,RX-11021598,N,Y,26.443001,-80.061098,https://www.redfin.com/FL/Delray-Beach/1225-S-...,Michael O'Connor,Douglas Elliman,Nicholas Malinosky,Douglas Elliman,Candace Friis,The Corcoran Group,Phillip Friis,The Corcoran Group,6,blue,Banyan House Condo at 1225 S Ocean Blvd #406 i...
6,PAST SALE,2024-11-12,Condo/Co-op,801 S Olive Ave #1604,West Palm Beach,FL,33401.0,1500000.0,2.0,2.0,One City Plaza Condominium,1475.0,NaN,2006.0,NaN,1017.0,1570.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/801-...,Beaches MLS,RX-10980984,N,Y,26.705820,-80.052488,https://www.redfin.com/FL/West-Palm-Beach/801-...,Kevin Oh,KW Reserve Palm Beach,None,None,Ginger Gowing,Luxury Real Estate Group LLC,None,None,7,blue,One City Plaza Condominium at 801 S Olive Av

In [47]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: N/a closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $735,000M to $12,500,000M
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Palm Beach County condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, North Palm Beach, Palm Beach Gardens, Boca Raton, Palm Beach, West Palm Beach, Delray Beach, Juno Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 62 condo sales totaling $50,160,200 million from Nov. 10th to Nov. 17th. The previous week, brokers closed 65 condo sales totaling $27,800,000.

Last week’s units sold for an average of $809,035, higher tha

In [72]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Palm-Beach/2-N-Breakers-Row-33480/unit-N-42/home/179496637


In [59]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Delray-Beach/1901-S-Ocean-Blvd-33483/unit-19/home/170176845


In [60]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Palm-Beach-Gardens/3630-Gardens-Pkwy-33410/unit-902C/home/42652445


In [51]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/North-Palm-Beach/100-Lakeshore-Dr-33408/unit-757/home/42481307


In [55]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Palm-Beach/2-N-Breakers-Row-33480/unit-N-42/home/179496637


In [56]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Palm-Beach-Gardens/3630-Gardens-Pkwy-33410/unit-902C/home/42652445


In [57]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-13,Condo/Co-op,2 N Breakers Row Unit N-42,Palm Beach,FL,33480.0,12500000.0,3.0,3.5,N/a,3129.0,NaN,1986.0,NaN,3995.0,15487.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Beaches MLS,RX-10939522,N,Y,26.717140,-80.034673,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Kevin Condon,"Sotheby's Intl. Realty, Inc.",None,None,Suzanne Trapani Frisbie,The Corcoran Group,None,None,1,orange,N/a at 2 N Breakers Row Unit N-42 in Palm Beach
1,PAST SALE,2024-11-14,Condo/Co-op,1901 S Ocean Blvd #19,Delray Beach,FL,33483.0,10796000.0,4.0,5.5,Ocean Delray Condo,3841.0,180.0,2021.0,NaN,2811.0,8097.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1901-S-...,Beaches MLS,RX-11021711,N,Y,26.437931,-80.062238,https://www.redfin.com/FL/Delray-Beach/1901-S-...,Jennifer Kilpatrick,The Corcoran Group,Mickel Averick,The Corcoran Group,Todd David Lipsich,Luxury Partners Realty,None,None,2,blue,Ocean Delray Condo at 1901 S Ocean Blvd #19 in...
2,PAST SALE,2024-11-14,Condo/Co-op,2100 S Ocean Blvd Unit 407s,Palm Beach,FL,33480.0,4050000.0,2.0,2.5,Twenty One Hundred Condo,2385.0,NaN,1980.0,NaN,1698.0,3034.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2100-S-Oc...,Beaches MLS,RX-11017997,N,Y,26.642292,-80.038181,https://www.redfin.com/FL/Palm-Beach/2100-S-Oc...,Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,Adam Zimmerman,Douglas Elliman (Palm Beach),Jacqueline Zimmerman,Douglas Elliman (Palm Beach),3,blue,Twenty One Hundred Condo at 2100 S Ocean Blvd ...
3,PAST SALE,2024-11-12,Condo/Co-op,700 Ocean Royale Way #1201,Juno Beach,FL,33408.0,2175000.0,3.0,3.5,Ocean Royale Condo,2936.0,NaN,2000.0,NaN,741.0,2900.0,Sold,NaN,NaN,https://www.redfin.com/FL/Juno-Beach/700-Ocean...,Beaches MLS,RX-11020844,N,Y,26.891690,-80.058145,https://www.redfin.com/FL/Juno-Beach/700-Ocean...,Holly Meyer Lucas,"Compass Florida, LLC (Jupiter)",Muriel Paton,"Compass Florida, LLC (Jupiter)",Holly Meyer Lucas,"Compass Florida, LLC (Jupiter)",Jenifer Greever,"Compass Florida, LLC (Jupiter)",4,blue,Ocean Royale Condo at 700 Ocean Royale Way #12...
4,PAST SALE,2024-11-13,Condo/Co-op,1225 S Ocean Blvd #306,Delray Beach,FL,33483.0,1850000.0,2.0,2.5,Banyan House Condo,1911.0,NaN,1967.0,NaN,968.0,1783.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1225-S-...,Beaches MLS,RX-11026097,N,Y,26.443001,-80.061098,https://www.redfin.com/FL/Delray-Beach/1225-S-...,Candace Friis,The Corcoran Group,Phillip Friis,The Corcoran Group,Candace Friis,The Corcoran Group,None,None,5,blue,Banyan House Condo at 1225 S Ocean Blvd #306 i...
5,PAST SALE,2024-11-12,Condo/Co-op,1225 S Ocean Blvd #406,Delray Beach,FL,33483.0,1725000.0,2.0,2.0,Banyan House Condo,1911.0,NaN,1967.0,NaN,903.0,1783.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1225-S-...,Beaches MLS,RX-11021598,N,Y,26.443001,-80.061098,https://www.redfin.com/FL/Delray-Beach/1225-S-...,Michael O'Connor,Douglas Elliman,Nicholas Malinosky,Douglas Elliman,Candace Friis,The Corcoran Group,Phillip Friis,The Corcoran Group,6,blue,Banyan House Condo at 1225 S Ocean Blvd #406 i...
6,PAST SALE,2024-11-12,Condo/Co-op,801 S Olive Ave #1604,West Palm Beach,FL,33401.0,1500000.0,2.0,2.0,One City Plaza Condominium,1475.0,NaN,2006.0,NaN,1017.0,1570.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/801-...,Beaches MLS,RX-10980984,N,Y,26.705820,-80.052488,https://www.redfin.com/FL/West-Palm-Beach/801-...,Kevin Oh,KW Reserve Palm Beach,None,None,Ginger Gowing,Luxury Real Estate Group LLC,None,None,7,blue,One City Plaza Condominium at 801 S Olive Av

## Time on Market Calculator

In [61]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 9, 17) ## List (Earlier) date
date2 = datetime(2024, 11, 14) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

58


## Clean CSV for Datawrapper Chart

In [62]:
chart_df = df_top_ten

In [63]:
chart_df = chart_df.fillna(" ")

In [64]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [65]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [66]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [67]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [68]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [69]:
csv_date_string = today.strftime("%m_%d_%Y")

In [70]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [71]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2024-11-13,2 N Breakers Row Unit N-42 in Palm Beach,"$12,500,000",3,3.5,"3,129",1986,"$3,995",Beaches MLS,RX-10939522,"Kevin Condon Sotheby's Intl. Realty, Inc.",Suzanne Trapani Frisbie The Corcoran Group
1,2024-11-14,1901 S Ocean Blvd #19 in Delray Beach,"$10,796,000",4,5.5,"3,841",2021,"$2,811",Beaches MLS,RX-11021711,Jennifer Kilpatrick The Corcoran Group Mickel ...,Todd David Lipsich Luxury Partners Realty
2,2024-11-14,2100 S Ocean Blvd Unit 407s in Palm Beach,"$4,050,000",2,2.5,"2,385",1980,"$1,698",Beaches MLS,RX-11017997,"Fern Fodiman Sotheby's Intl. Realty, Inc.",Adam Zimmerman Douglas Elliman (Palm Beach) Ja...
3,2024-11-12,700 Ocean Royale Way #1201 in Juno Beach,"$2,175,000",3,3.5,"2,936",2000,$741,Beaches MLS,RX-11020844,"Holly Meyer Lucas Compass Florida, LLC (Jupite...","Holly Meyer Lucas Compass Florida, LLC (Jupite..."
4,2024-11-13,1225 S Ocean Blvd #306 in Delray Beach,"$1,850,000",2,2.5,"1,911",1967,$968,Beaches MLS,RX-11026097,Candace Friis The Corcoran Group Phillip Friis...,Candace Friis The Corcoran Group
5,2024-11-12,1225 S Ocean Blvd #406 in Delray Beach,"$1,725,000",2,2.0,"1,911",1967,$903,Beaches MLS,RX-11021598,Michael O'Connor Douglas Elliman Nicholas Mali...,Candace Friis The Corcoran Group Phillip Friis...
6,2024-11-12,801 S Olive Ave #1604 in West Palm Beach,"$1,500,000",2,2.0,"1,475",2006,"$1,017",Beaches MLS,RX-10980984,Kevin Oh KW Reserve Palm Beach,Ginger Gowing Luxury Real Estate Group LLC
7,2024-11-14,2000 S Ocean Blvd Unit 5-E in Boca Raton,"$1,140,000",2,2.0,"1,236",1971,$922,Beaches MLS,RX-11022495,Gregory Lynn Lynn Realty Group Charli Lynn Lyn...,Lemore Zausner Engel & Voelkers Delray Beach M...
8,2024-11-14,100 Lakeshore Dr #757 in North Palm Beach,"$1,090,000",2,2.5,"2,646",1982,$412,Beaches MLS,RX-10990950,"Diana Casper Illustrated Properties, LLC",Kim-Renee Roberts Water Club Realty LLC
9,2024-11-12,3630 Gardens Pkwy Unit 902c in Palm Beach Gardens,"$735,000",2,2.5,"1,870",2007,$393,Beaches MLS,RX-11029474,Hailey Desser EXP Realty LLC,Robert Pasquale Keller Williams Realty Jupiter...
